#This code is for scraping rare diseases from NIH genetic and rare disease database (GARD)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup
import time
import json 

service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument("--headless")  
driver = webdriver.Chrome(service=service, options=options)

diseases_data = []

for page in range(1, 615):  
    url = f"https://rarediseases.info.nih.gov/diseases?page={page}&search=&letter=&category="
    #print(url)
    
    driver.get(url)
    time.sleep(1)  
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    disease_divs = soup.find_all("div", class_="result mb-5 cursor-pointer")

    for div in disease_divs:
        name_tag = div.find("h5")
        disease_name = name_tag.text.strip() if name_tag else "N/A"
        
        link_tag = div.find("a")
        disease_url = f"https://rarediseases.info.nih.gov/{link_tag['href']}" if link_tag else "N/A"

        other_names_tag = div.find("p")
        other_names = "N/A"
        if other_names_tag:
            text = other_names_tag.text.strip()
            if "Other names:" in text:
                other_names = text.split("Other names:")[1].strip()
        
        diseases_data.append({
            "disease_name": disease_name,
            "disease_url": disease_url,
            "other_names": other_names
        })
    
    
    
driver.quit()
#print(diseases_data)


In [ ]:
print(len(diseases_data))

In [ ]:
with open("diseases.json", "w", encoding="utf-8") as json_file:
    json.dump(diseases_data, json_file, indent=4, ensure_ascii=False)